# Cybersecurity AI Application Architecture

```mermaid
graph TD
    %% Main Application
    A[cybersec_app.py] --> B[Agent Layer]
    
    %% Agent connects to all modules
    B --> C1[Bayesian Analysis Module]
    B --> C2[Markov Decision Process Module]
    B --> C4[Service Impact Analysis Module]
    
    %% Bayesian Module Components
    C1 --> D1[bayesian_ids.py]
    C1 --> D2[bayesian_network.py]
    
    %% MDP Module
    C2 --> D3[markov_process.py]
    
    %% Service Impact Analysis Module
    C4 --> D5[service_impact.py]
    
    %% Data Storage
    E[Data Storage Layer]
    D1 --> E
    D2 --> E
    D3 --> E
    D5 --> E
    
    %% Module Descriptions
    classDef module fill:#f9f,stroke:#333,stroke-width:2px
    classDef component fill:#bbf,stroke:#333,stroke-width:1px
    
    class A,B module
    class C1,C2,C4 module
    class D1,D2,D3,D5,E component
```

## Module Descriptions

### Agent Layer
- Implements the intelligent agent from Week 1
- Uses search algorithms to detect and respond to changes
- Acts as a simple reflex agent that processes inputs and determines appropriate actions

### Bayesian Analysis Module
- **bayesian_ids.py**: Simple Bayesian analysis for intrusion detection systems
- **bayesian_network.py**: Extended Bayesian network model incorporating multiple security factors

### Markov Decision Process Module
- **markov_process.py**: MDP implementation for optimal security response policies

### Service Impact Analysis Module
- **service_impact.py**: DFS/BFS search algorithms for CI/CD pipeline optimization

### Data Storage Layer
- Persists analysis results and configurations
- Enables sharing of information between modules

## Data Flow

1. Agent perceives the environment through sensors
2. Agent selects appropriate module based on perceived data
3. Selected module processes the input using AI algorithms
4. Results are used by the agent to determine actions
5. Key insights and configurations stored in data layer

This architecture implements an agent-based approach where the agent uses different AI techniques to monitor and respond to cybersecurity events. The agent replaces the traditional UI, providing more automated operation based on the search algorithms developed in Week 1.


# Running the Cybersecurity AI Application

This notebook allows you to run and test the Cybersecurity AI application directly. Below are the code cells needed to initialize and run the application.

## 1. Import Required Modules

In [18]:
# Import necessary modules
import time
import random
import sys
import os

# Ensure the module paths are correct
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the Agent
from agent import Agent

## 2. Define Mock Data Generation Function

In [19]:
def generate_mock_data(event_type):
    """Generate mock data for different event types for simulation"""
    if event_type == 'login_attempt':
        return {
            'username': random.choice(['admin', 'user1', 'guest']),
            'source_ip': f'192.168.1.{random.randint(1, 254)}' if random.random() > 0.3 else f'10.0.0.{random.randint(1, 254)}',
            'timestamp': time.time(),
            'successful': random.choice([True, False])
        }
    elif event_type == 'service_change':
        # Randomly include a known malicious commit 5% of the time
        if random.random() < 0.05:
            commit_sha = 'a1b2c3d4e5f6a1b2c3d4e5f6a1b2c3d4e5f6a1b2'  # Known malicious
        else:
            commit_sha = ''.join(random.choices('abcdef1234567890', k=40))
            
        return {
            'service': random.choice(['Auth Service', 'API Gateway', 'User Service', 'unknown_service']),
            'change_type': random.choice(['code_update', 'configuration_change', 'dependency_update']),
            'commit_sha': commit_sha
        }
    elif event_type == 'git_activity':
        # Generate mock git activity events
        git_event_types = ['push', 'clone', 'branch']
        selected_event = random.choice(git_event_types)
        
        # Common fields
        event_data = {
            'event_type': selected_event,
            'repo_name': random.choice(['main-service', 'auth-service', 'payment-api', 'user-interface']),
            'user': random.choice(['developer1', 'developer2', 'admin', 'unknown_user']),
            'timestamp': time.time(),
            'ip_address': f'192.168.1.{random.randint(1, 254)}' if random.random() > 0.2 else f'203.0.113.{random.randint(1, 254)}'
        }
        
        # Add event-specific fields
        if selected_event == 'push':
            # Generate 1-12 commits
            num_commits = random.randint(1, 12)
            event_data['commits'] = []
            for _ in range(num_commits):
                commit = {
                    'commit_sha': ''.join(random.choices('abcdef1234567890', k=40)),
                    'message': random.choice(['Update README', 'Fix bug in auth module', 'Add new feature', 'Refactor code', 'Update dependencies']),
                    'files_changed': []
                }
                # Add 1-5 changed files
                num_files = random.randint(1, 5)
                file_paths = [
                    'src/main.py', 'config/settings.json', 'docs/README.md',
                    'src/auth/login.py', 'tests/test_api.py', 'security/encryption.py',
                    'src/models/user.py', '.env.example', 'config/secrets.py'
                ]
                commit['files_changed'] = random.sample(file_paths, min(num_files, len(file_paths)))
                event_data['commits'].append(commit)
                
            # Small chance of being a force push
            event_data['force_push'] = random.random() < 0.1
            
        elif selected_event == 'branch':
            # Branch creation or deletion
            event_data['action'] = random.choice(['created', 'deleted'])
            branch_prefixes = ['feature/', 'bugfix/', 'hotfix/', 'release/', 'temp/', 'test/']
            event_data['branch_name'] = random.choice(branch_prefixes) + random.choice(['auth-update', 'payment-fix', 'ui-redesign', 'admin-tools', 'security-patch'])
            
            # For deleted branches, add age
            if event_data['action'] == 'deleted':
                event_data['branch_age_hours'] = random.uniform(0.5, 120)  # 30 minutes to 5 days
        
        # Clone events have all necessary data already
            
        return event_data
    else:  # network_traffic
        return {
            'source_ip': f'192.168.1.{random.randint(1, 254)}' if random.random() > 0.2 else f'8.8.8.{random.randint(1, 254)}',
            'destination_ip': f'10.0.0.{random.randint(1, 254)}',
            'protocol': random.choice(['TCP', 'UDP', 'HTTP']),
            'packet_count': random.randint(10, 1000)
        }

## 3. Initialize the Agent

In [20]:
# Create and initialize the agent
print("Starting Cybersecurity AI Application...")
print("Initializing agent and modules...")

agent = Agent()
print("Agent initialization complete.")

Starting Cybersecurity AI Application...
Initializing agent and modules...
Agent initialized with all modules
Agent initialization complete.


## 4. Run Test Cases

### 4.1 Test Login Attempt Detection

In [21]:
# Test with a suspicious login attempt (admin, failed login)
suspicious_login = {
    'username': 'admin',
    'source_ip': '10.0.0.25',  # External IP
    'timestamp': time.time(),
    'successful': False  # Failed login
}

print("--- Testing Login Attempt Detection ---")
print(f"Event data: {suspicious_login}")
action = agent.process_event('login_attempt', suspicious_login)
print(f"Agent response: {action}\n")

--- Testing Login Attempt Detection ---
Event data: {'username': 'admin', 'source_ip': '10.0.0.25', 'timestamp': 1741800069.5536919, 'successful': False}
Agent response: MEDIUM ALERT: Suspicious activity detected (confidence: 0.66)



### 4.2 Test Service Change Impact Analysis

In [22]:
# Test with a suspicious service change (known malicious commit)
suspicious_change = {
    'service': 'API Gateway',
    'change_type': 'configuration_change',
    'commit_sha': 'a1b2c3d4e5f6a1b2c3d4e5f6a1b2c3d4e5f6a1b2'  # Known malicious
}

print("--- Testing Service Change Impact Analysis ---")
print(f"Event data: {suspicious_change}")
action = agent.process_event('service_change', suspicious_change)
print(f"Agent response: {action}\n")

--- Testing Service Change Impact Analysis ---
Event data: {'service': 'API Gateway', 'change_type': 'configuration_change', 'commit_sha': 'a1b2c3d4e5f6a1b2c3d4e5f6a1b2c3d4e5f6a1b2'}
Agent response: SECURITY ALERT: Suspicious commit detected in API Gateway! Impacts 8 services. Recommend immediate review.



### 4.3 Test Network Traffic Analysis

In [23]:
# Test with suspicious network traffic (high volume, external source)
suspicious_traffic = {
    'source_ip': '8.8.8.45',  # External IP
    'destination_ip': '10.0.0.10',
    'protocol': 'UDP',  # More suspicious than HTTP
    'packet_count': 950  # High volume
}

print("--- Testing Network Traffic Analysis ---")
print(f"Event data: {suspicious_traffic}")
action = agent.process_event('network_traffic', suspicious_traffic)
print(f"Agent response: {action}\n")

--- Testing Network Traffic Analysis ---
Event data: {'source_ip': '8.8.8.45', 'destination_ip': '10.0.0.10', 'protocol': 'UDP', 'packet_count': 950}
Agent response: NETWORK ACTION: Flag for manual review (risk level: 0.58)



### 4.4 Test Git Activity Monitoring

In [24]:
# Test with suspicious git activity
suspicious_git_activity = {
    'event_type': 'push',
    'repo_name': 'auth-service',
    'user': 'developer1',
    'timestamp': time.time() - 12 * 3600,  # 12 hours ago (middle of night)
    'ip_address': '203.0.113.45',  # External IP
    'commits': [
        {
            'commit_sha': 'abc123def456abc123def456abc123def456abc1',
            'message': 'quick fix',
            'files_changed': ['security/auth.py', 'config/secrets.py', '.env']
        }
    ],
    'force_push': True
}

print("--- Testing Git Activity Monitoring ---")
print(f"Event data: {suspicious_git_activity}")
action = agent.process_event('git_activity', suspicious_git_activity)
print(f"Agent response: {action}\n")

--- Testing Git Activity Monitoring ---
Event data: {'event_type': 'push', 'repo_name': 'auth-service', 'user': 'developer1', 'timestamp': 1741756869.570909, 'ip_address': '203.0.113.45', 'commits': [{'commit_sha': 'abc123def456abc123def456abc123def456abc1', 'message': 'quick fix', 'files_changed': ['security/auth.py', 'config/secrets.py', '.env']}], 'force_push': True}
Agent response: Unknown event type, no action taken



## 5. Run a Full Simulation

In [25]:
# Run a full simulation with random events
print("\nRunning full cybersecurity monitoring simulation:")
for i in range(10):
    print(f"\n--- Monitoring Cycle {i+1} ---")
    
    # Randomly generate an event type
    event_type = random.choice(['login_attempt', 'service_change', 'network_traffic', 'git_activity'])
    event_data = generate_mock_data(event_type)
    
    print(f"Event detected: {event_type}")
    print(f"Event data: {event_data}")
    
    # Agent perceives and acts on the event
    action = agent.process_event(event_type, event_data)
    
    print(f"Agent response: {action}")

print("\nCybersecurity monitoring simulation completed.")


Running full cybersecurity monitoring simulation:

--- Monitoring Cycle 1 ---
Event detected: login_attempt
Event data: {'username': 'guest', 'source_ip': '10.0.0.167', 'timestamp': 1741800069.576135, 'successful': True}
Agent response: LOW ALERT: Normal login activity (confidence: 0.78)

--- Monitoring Cycle 2 ---
Event detected: service_change
Event data: {'service': 'unknown_service', 'change_type': 'code_update', 'commit_sha': 'b232b68f3e64583c76ffab7a952493134d0a16c4'}
Agent response: SECURITY ALERT: Suspicious commit detected in unknown_service! Impacts 0 services. Recommend immediate review.

--- Monitoring Cycle 3 ---
Event detected: service_change
Event data: {'service': 'unknown_service', 'change_type': 'code_update', 'commit_sha': 'baf51bb0961adc9b704f4da696595ecb3a14d583'}
Agent response: SECURITY ALERT: Suspicious commit detected in unknown_service! Impacts 0 services. Recommend immediate review.

--- Monitoring Cycle 4 ---
Event detected: git_activity
Event data: {'event

## 6. Review Data Storage

In [26]:
# Review data that has been stored during the simulation
print("\n--- Data Storage Contents ---")

# Check login analysis data
login_data = agent.data_storage.retrieve('login_analysis')
print(f"\nLogin Analysis Records: {len(login_data)}")
if login_data:
    print(f"Latest record: {login_data[-1]}")

# Check service impact analysis data
service_data = agent.data_storage.retrieve('service_impact_analysis')
print(f"\nService Impact Analysis Records: {len(service_data)}")
if service_data:
    print(f"Latest record: {service_data[-1]}")

# Check network analysis data
network_data = agent.data_storage.retrieve('network_analysis')
print(f"\nNetwork Analysis Records: {len(network_data)}")
if network_data:
    print(f"Latest record: {network_data[-1]}")

# Check MDP decision data
mdp_data = agent.data_storage.retrieve('mdp_decision')
print(f"\nMDP Decision Records: {len(mdp_data)}")
if mdp_data:
    print(f"Latest record: {mdp_data[-1]}")

# Check git activity analysis data
git_data = agent.data_storage.retrieve('git_activity_analysis')
print(f"\nGit Activity Analysis Records: {len(git_data)}")
if git_data:
    print(f"Latest record: {git_data[-1]}")


--- Data Storage Contents ---

Login Analysis Records: 3
Latest record: {'timestamp': 1741800069.5763788, 'username': 'user1', 'source_ip': '192.168.1.245', 'attack_probability': 0.15517241379310345}

Service Impact Analysis Records: 1
Latest record: {'changed_service': 'API Gateway', 'change_type': 'configuration_change', 'commit_sha': 'a1b2c3d4e5f6a1b2c3d4e5f6a1b2c3d4e5f6a1b2', 'impacted_services': ['Email Service', 'Notification Service', 'Payment Service', 'API Gateway', 'Inventory Service', 'User Service', 'Profile Service', 'Order Service'], 'commit_risk': 'high'}

Network Analysis Records: 1
Latest record: {'traffic_volume': 'high', 'protocol': 'UDP', 'internal_source': False, 'risk_probability': 0.58}

MDP Decision Records: 1
Latest record: {'state': 'network_anomaly', 'risk_level': 0.58, 'risk_category': 'medium', 'action': 'Flag for manual review'}

Git Activity Analysis Records: 0
